# LSTM 

## Importer et préparer des données

Nous utiliserons des séquences remplies, ce qui fera que notre RNN ne traitera que les éléments non remplis de notre séquence, et pour tout élément rempli, la sortie sera un tenseur nul. Pour utiliser des séquences rembourrées, nous devons indiquer au RNN la longueur des séquences réelles. Nous faisons cela en définissant include_lengths = True pour notre champ TEXT. Cela fera que batch.text sera maintenant un tuple avec le premier élément étant notre phrase (un tenseur numérisé qui a été complété) et le deuxième élément étant les longueurs réelles de nos phrases.

In [1]:
import pandas as pd
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

TEXT = data.Field(sequential=True,lower=True, tokenize = 'spacy', preprocessing = generate_bigrams, 
                  include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)
train_data, valid_data, test_data = data.TabularDataset.splits(
        path='./data/', train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')


Taille des données train: 18163
Taille des données de validation: 2270
Taille des données test: 2271


## Vocabulaire



Vient ensuite l'utilisation de word embeding pré-entraînées. Maintenant, au lieu d'avoir nos embeddings de mots initialisés au hasard, ils sont initialisés avec ces vecteurs pré-entraînés. Nous obtenons ces vecteurs simplement en spécifiant quels vecteurs nous voulons et en les passant comme argument à build_vocab. TorchText gère le téléchargement des vecteurs et les associe aux mots corrects de notre vocabulaire.

Ici, nous allons utiliser les "vecteurs" glove.6B.100d. Glove est l'algorithme utilisé pour calculer les vecteurs, allez ici pour en savoir plus. 6B indique que ces vecteurs ont été entraînés sur 6 milliards de jetons et 100d indique que ces vecteurs sont 100 -dimensionnelle.

In [2]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

## Itérateur

Pour les séquences rembourrées emballées, tous les tenseurs d'un lot doivent être triés par leurs longueurs. Ceci est géré dans l'itérateur en définissant sort_within_batch = True.

In [3]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device,sort_key=lambda x: len(x.text),
    sort_within_batch = True)

## Modèle

Dans ce modèle, on a :

 - LSTM au lieu de RNN. En effet, les RNN standard souffrent du problème de disparition du gradient. Les LSTM surmontent ce problème en ayant un état récurrent supplémentaire appelé cellule, $ c $  qui peut être considéré comme la «mémoire» du LSTM et en utilisant plusieurs portes (gates) qui contrôlent le flux d'informations dans et hors de la mémoire. Nous pouvons simplement considérer le LSTM comme une fonction de $ x_t $, $ h_t $ et $ c_t $, au lieu de seulement $ x_t $ et $ h_t $. $$(h_t, c_t) = \text{LSTM}(x_t, h_t, c_t)$$L'état initial de la cellule, $ c_0 $, comme l'état caché initial est initialisé à un tenseur égal à zéro. Cependant, la prédiction de sentiment n'est faite qu'en utilisant l'état caché final, pas l'état final de la cellule, c'est-à-dire $ \hat {y} = f (h_T) $.
 - Bidirectionnel : en plus d'avoir un LSTM traitant les mots de la phrase du premier au dernier (un LSTM forward), nous avons un deuxième LSTM traitant les mots de la phrase du dernier au premier (un LSTM backward). Au pas de temps $t$, le LSTM forward traite le mot $ x_t $, et le LSTM backward traite le mot $ x_ {T-t + 1} $.
 - Multi-couches : l'idée est que nous ajoutons des LSTM supplémentaires en plus du LSTM standard initial, où chaque LSTM ajouté est une autre couche. La sortie d'état caché par le premier LSTM (inférieur) au pas de temps $t$ sera l'entrée du LSTM au-dessus de lui au pas de temps $ t $. La prédiction est alors faite à partir de l'état caché final de la couche finale (la plus haute).
 - Régularisation : plus on a de paramètres dans notre modèle, plus la probabilité que notre modèle se sur-ajustement est élevée (mémoriser les données d'entraînement, provoquant une erreur d'entraînement faible mais une erreur de validation / test élevée, c'est-à-dire une mauvaise généralisation à de nouveaux exemples). Pour lutter contre cela, nous utilisons la régularisation. Plus précisément, nous utilisons une méthode de régularisation appelée dropout. Le dropout fonctionne en supprimant aléatoirement (mise à 0) les neurones dans une couche lors d'un passage vers l'avant.
 - Avant de transmettre les embeddings au LSTM, nous devons les emballer, ce que nous faisons avec nn.utils.rnn.packed_padded_sequence. Cela obligera notre LSTM à ne traiter que les éléments non rembourrés de notre séquence. Le LSTM retournera alors packed_output (une séquence condensée) ainsi que les états caché et cellule (qui sont tous deux des tenseurs). Sans séquences remplies remplies, masqué et cellule sont des tenseurs du dernier élément de la séquence, qui sera très probablement un token de remplissage, mais lorsque vous utilisez des séquences remplies de remplissage, ils proviennent tous deux du dernier élément non rempli de la séquence. Nous décompressons ensuite la séquence de sortie, avec nn.utils.rnn.pad_packed_sequence, pour la transformer d'une séquence compressée en un tenseur. Les éléments de sortie des tokens de remplissage seront des tenseurs nuls (tenseurs où chaque élément est nul). Habituellement, nous n'avons à décompresser la sortie que si nous voulons l'utiliser plus tard dans le modèle. 
 
 
L'état caché final, caché, a une forme de **[nombre de couches * nombre de directions, taille de lot, caché dim]**. Ceux-ci sont classés: **[forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n]**. Comme nous voulons les états cachés avant et arrière de la couche finale (supérieure), nous obtenons les deux couches cachées supérieures de la première dimension, cachées [-2,:,:] et cachées [-1,:,:], et les concaténons ensemble avant de les transmettre au calque linéaire (après application de la suppression).
 


In [4]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

Comme auparavant, nous allons créer une instance de notre classe RNN, avec les nouveaux paramètres et arguments pour le nombre de couches, la bidirectionnalité et la probabilité de dropout.

Pour garantir que les vecteurs pré-entraînés peuvent être chargés dans le modèle, EMBEDDING_DIM doit être égal à celui des vecteurs GloVe pré-entraînés chargés précédemment.

Nous obtenons notre index de jeton de pad à partir du vocabulaire, obtenant la chaîne réelle représentant le jeton de pad à partir de l'attribut pad_token du champ, qui est <pad> par défaut.

In [5]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


Le dernier ajout consiste à copier les embeddings de mots pré-entraînées que nous avons chargées précédemment dans la couche d'embedding de notre modèle.

Nous récupérons les embeddings à partir du vocabulaire de Field et vérifions qu'elles sont de la bonne taille, **[taille du vocabulaire,dim embedding]**

Comme nos tokens <unk> et <pad> ne sont pas dans le vocabulaire pré-entraîné, ils ont été initialisés en utilisant unk_init (une distribution $ \mathcal {N} (0,1) $) lors de la construction de notre vocabulaire. Il est préférable de les initialiser tous les deux à zéros pour indiquer explicitement à notre modèle que, au départ, ils ne sont pas pertinents pour déterminer le sentiment.


In [6]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

torch.Size([25002, 100])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 2.2529,  0.8734, -0.3372,  ...,  0.8701, -0.1612,  0.4326],
        [-1.5441,  1.0105,  0.9192,  ...,  1.0422,  0.3977, -0.7357],
        [-0.4597,  1.4756,  1.1808,  ...,  1.5027, -0.8486,  0.6017]])


## Métriques

In [7]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def precision(preds, y):
    '''
    Retourne la précision
    '''
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
       
    precision = tp / (tp + fp)
    
    return precision

def recall(preds, y):
    '''
    Retourne le recall
    '''
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float()       
    
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    recall = tp / (tp + fn)
    return recall


def f1_loss(preds, y):
    '''
    Retourne le score F1
    '''  
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    
    f1 = 2* (precision*recall) / (precision + recall)
    return f1

## Optimiseur et fonction de perte

In [8]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)


## Entraîner le modèle

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    


model.load_state_dict(torch.load('tut2-model.pt'))

## Tester le modèle

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Notre fonction predict_sentiment fait plusieurs choses:

 - définit le modèle en mode d'évaluation
 - tokenise la phrase, c'est-à-dire la divise d'une chaîne brute en une liste de jetons
 - indexe les jetons en les convertissant en leur représentation entière à partir de notre vocabulaire
 - obtient la longueur de notre séquence
 - convertit les index, qui sont une liste Python en un tenseur PyTorch
 - ajouter une dimension de lot en relâchant
 - convertit la longueur en un tenseur
 - écrase la prédiction de sortie d'un nombre réel compris entre 0 et 1 avec la fonction sigmoïde
 - convertit le tenseur contenant une valeur unique en un entier avec la méthode item ()

Nous nous attendons à ce que les avis avec un sentiment négatif renvoient une valeur proche de 0 et les avis positifs renvoient une valeur proche de 1.


In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is great")

In [ ]:
predict_sentiment(model, "This film is terrible")